In [ ]:
import sys
sys.path.insert(0, '../..')
from util import carregamento_dados as cd, ferramentas_grafos as fg
import pandas as pd
from collections import defaultdict

In [2]:
# Carrega os 3 arquivos principais
dump_path = '../../pickles/licitacoes/'
relacoes_entre_cnpjs = cd.salvar_relacoes_entre_cnpjs()
informacoes_licitacoes = cd.salvar_informacoes_licitacoes()
cnpjs_por_licitacao = cd.salvar_cnpjs_por_licitacao()

# Dicionários necessários carregados por meio de funções implementadas
# externamente.
d_relacoes = cd.cnpjs_relacionados_por_cnpj(relacoes_entre_cnpjs)
d_licitacoes = cd.cnpjs_por_licitacao(cnpjs_por_licitacao)

In [3]:
dados_licitacao = cnpjs_por_licitacao.values
licitacoes = [licitacao for licitacao, _ in dados_licitacao]

d = {
    licitacao: {
        'cnpjs' : [],
        'ano' : None,
        'municipio': None,
        'modalidade': None,
        'grafo': None,
        'cliques': None
    } for licitacao in licitacoes
}

for licitacao, cnpj_licitante in dados_licitacao:
    d[licitacao]['cnpjs'].append(cnpj_licitante)

In [4]:
for licitacao in licitacoes:
    grafo = fg.gera_grafo_licitacao(
        licitacao, d_relacoes, d_licitacoes
    )
    cliques = fg.lista_cliques(grafo)

    d[licitacao]['grafo'] = grafo
    d[licitacao]['cliques'] = cliques

In [5]:
c = defaultdict(dict)
clique_id = 0
for licitacao in licitacoes:
    for clique in d[licitacao]['cliques']:
        if len(clique) >= 2:
            c[clique_id] = {
                'cnpjs' : d[licitacao]['cnpjs'],
                'licitacao_id' : licitacao
            }
            clique_id += 1